In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_wi.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,945699,1040127103,4/9/20,563,PPP,"WISCONSIN KENWORTH, LLC","5100 Eastpark Blvd, Suite 210 0.0",Madison,WI,53718.0,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,NaN
1,945700,1299787308,4/28/20,563,PPP,TOTALMED INC.,221 W. COLLEGE AVE Floor 2,APPLETON,WI,54911.0,...,999 Other,5613 Employment Services,5613.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,NaN
2,945701,1525827110,4/10/20,563,PPP,COURTESY CORPORATION,2700 National Drive Ste 100,ONALASKA,WI,54650.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,NaN
3,945702,2572157107,4/10/20,563,PPP,AIR WISCONSIN AIRLINES LLC,"W6390 Challenger Dr, Suite 203 0.0",Appleton,WI,54914.0,...,999 Other,4811 Scheduled Air Transportation,4811.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,NaN
4,945703,3943447203,4/27/20,563,PPP,"FOREMOST FARMS USA, COOPERATIVE",10889 E PENNY LN,BARABOO,WI,53913.0,...,311 Food,3115 Dairy Product Manufacturing,3115.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17349,963048,9282518304,1/30/21,563,PPS,RESOURCE MACHINING & WELDING CORP.,W229N5065 Duplainville Rd,Pewaukee,WI,53072.0,...,332 Fabricated Metal Products,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,NaN
17350,963049,9361087700,5/1/20,563,PPP,"KORTENDICK HARDWARE, INC.",3806 DOUGLAS AVENUE,RACINE,WI,53402.0,...,999 Other,4441 Building Material and Supplies Dealers,4441.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,NaN
17351,963050,9411997102,4/15/20,563,PPP,HIGHLAND PARK FAMILY DENTISTRY SC,6101 W VLIET ST,WAUWATOSA,WI,53213.0,...,999 Other,6212 Offices of Dentists,6212.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,NaN
17352,963051,9453378305,1/30/21,563,PPS,GRUETZ INC,E5690 County Road D,Menomonie,WI,54751.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
3247,5637407007,"320 10th Ave., PO Box 96, BALDWIN, WI",NaN,NaN
16328,1404407106,"715 MORGAN ST PO BOX 247, CLYMAN, WI",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,1040127103,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690
1,1299787308,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348
2,1525827110,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984
3,2572157107,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351
4,3943447203,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WI_fips_scraped.csv")


In [2]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WI_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,945699,945699,945699,1040127103,4/9/20,563,PPP,"WISCONSIN KENWORTH, LLC","5100 Eastpark Blvd, Suite 210 0.0",...,999 Other,8111 Automotive Repair and Maintenance,8111.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14
1,1,945700,945700,945700,1299787308,4/28/20,563,PPP,TOTALMED INC.,221 W. COLLEGE AVE Floor 2,...,999 Other,5613 Employment Services,5613.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14
2,2,945701,945701,945701,1525827110,4/10/20,563,PPP,COURTESY CORPORATION,2700 National Drive Ste 100,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14
3,3,945702,945702,945702,2572157107,4/10/20,563,PPP,AIR WISCONSIN AIRLINES LLC,"W6390 Challenger Dr, Suite 203 0.0",...,999 Other,4811 Scheduled Air Transportation,4811.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14
4,4,945703,945703,945703,3943447203,4/27/20,563,PPP,"FOREMOST FARMS USA, COOPERATIVE",10889 E PENNY LN,...,311 Food,3115 Dairy Product Manufacturing,3115.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [6]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1040127103,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14
1,1299787308,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14
2,1525827110,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14
3,2572157107,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14
4,3943447203,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14
...,...,...,...,...,...
17349,9282518304,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,NaN
17350,9361087700,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,NaN
17351,9411997102,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,NaN
17352,9453378305,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,NaN


In [7]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() 

LoanNumber    12822
full_add      12822
Lat           12822
Long          12822
FIPS_z            0
dtype: int64

<h4>FIPS Script

In [8]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [9]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('55')] #WI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


  0%|          | 0/12822 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [10]:
geo_id_list =geo_id 

Length Check

In [11]:
print(len(geo_id_list))

12822


Matching FIPS to DataFrame

In [12]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:4537]
#if FIPS available
fips_fill = fips_check_scraped_null

In [13]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_24922/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
547,4648077103,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,[]
1641,2925978305,"215 Front St, Clinton, WI",42.410926,-71.689619,[]
1643,7736337004,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,[]
1712,6987067006,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,[]
2027,9739697001,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,[]
...,...,...,...,...,...
17349,9282518304,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,[551332033044000]
17350,9361087700,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,[551010014013000]
17351,9411997102,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,[550790912001000]
17352,9453378305,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,[550339703002036]


Remove brackets

In [14]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_24922/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
547,4648077103,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,NaN
1641,2925978305,"215 Front St, Clinton, WI",42.410926,-71.689619,NaN
1643,7736337004,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,NaN
1712,6987067006,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,NaN
2027,9739697001,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,NaN
...,...,...,...,...,...
17349,9282518304,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,551332033044000
17350,9361087700,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,551010014013000
17351,9411997102,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,550790912001000
17352,9453378305,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,550339703002036


In [15]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
547,4648077103,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,NaN
1641,2925978305,"215 Front St, Clinton, WI",42.410926,-71.689619,NaN
1643,7736337004,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,NaN
1712,6987067006,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,NaN
2027,9739697001,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,NaN
2151,9170817005,"7213 HWY 41 0.0, Caledonia, WI",25.763107,-80.312898,NaN
2659,2688497106,"12571 OLIVER AVE S STE 100, MILWAUKEE, WI",44.776233,-93.305704,NaN
8676,4612047005,"5074 HWY 51 NORTH, HARSHAW, WI",34.398029,-89.927004,NaN
10304,4769537008,"600 North First Street, CAMERON, WI",37.346986,-121.898560,NaN
12627,6978328300,"578 Allen St, Amherst, WI",42.091848,-72.543053,NaN


Parse for Merge

In [16]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
547,4648077103,NaN
1641,2925978305,NaN
1643,7736337004,NaN
1712,6987067006,NaN
2027,9739697001,NaN
...,...,...
17349,9282518304,551332033044000
17350,9361087700,551010014013000
17351,9411997102,550790912001000
17352,9453378305,550339703002036


Read in base data

In [20]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_wi.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WI_fips_scraped.csv") 
#fips_check = fips_check.drop([ "Unnamed: 0.1.1.1.1.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,945699,945699,945699,1040127103,4/9/20,563,PPP,"WISCONSIN KENWORTH, LLC","5100 Eastpark Blvd, Suite 210 0.0",...,999 Other,8111 Automotive Repair and Maintenance,8111.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14
1,1,945700,945700,945700,1299787308,4/28/20,563,PPP,TOTALMED INC.,221 W. COLLEGE AVE Floor 2,...,999 Other,5613 Employment Services,5613.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14
2,2,945701,945701,945701,1525827110,4/10/20,563,PPP,COURTESY CORPORATION,2700 National Drive Ste 100,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14
3,3,945702,945702,945702,2572157107,4/10/20,563,PPP,AIR WISCONSIN AIRLINES LLC,"W6390 Challenger Dr, Suite 203 0.0",...,999 Other,4811 Scheduled Air Transportation,4811.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14
4,4,945703,945703,945703,3943447203,4/27/20,563,PPP,"FOREMOST FARMS USA, COOPERATIVE",10889 E PENNY LN,...,311 Food,3115 Dairy Product Manufacturing,3115.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14


Merge with MainFrame

In [21]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,945699,945699,945699,1040127103,4/9/20,563,PPP,"WISCONSIN KENWORTH, LLC","5100 Eastpark Blvd, Suite 210 0.0",...,8111 Automotive Repair and Maintenance,8111.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14,NaN
1,1,945700,945700,945700,1299787308,4/28/20,563,PPP,TOTALMED INC.,221 W. COLLEGE AVE Floor 2,...,5613 Employment Services,5613.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14,NaN
2,2,945701,945701,945701,1525827110,4/10/20,563,PPP,COURTESY CORPORATION,2700 National Drive Ste 100,...,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14,NaN
3,3,945702,945702,945702,2572157107,4/10/20,563,PPP,AIR WISCONSIN AIRLINES LLC,"W6390 Challenger Dr, Suite 203 0.0",...,4811 Scheduled Air Transportation,4811.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14,NaN
4,4,945703,945703,945703,3943447203,4/27/20,563,PPP,"FOREMOST FARMS USA, COOPERATIVE",10889 E PENNY LN,...,3115 Dairy Product Manufacturing,3115.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17349,17349,963048,963048,963048,9282518304,1/30/21,563,PPS,RESOURCE MACHINING & WELDING CORP.,W229N5065 Duplainville Rd,...,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,NaN,551332033044000
17350,17350,963049,963049,963049,9361087700,5/1/20,563,PPP,"KORTENDICK HARDWARE, INC.",3806 DOUGLAS AVENUE,...,4441 Building Material and Supplies Dealers,4441.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,NaN,551010014013000
17351,17351,963050,963050,963050,9411997102,4/15/20,563,PPP,HIGHLAND PARK FAMILY DENTISTRY SC,6101 W VLIET ST,...,6212 Offices of Dentists,6212.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,NaN,550790912001000
17352,17352,963051,963051,963051,9453378305,1/30/21,563,PPS,GRUETZ INC,E5690 County Road D,...,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,NaN,550339703002036


<b>Fill FIPS Columns - if FIPS file

In [22]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,945699,945699,945699,1040127103,4/9/20,563,PPP,"WISCONSIN KENWORTH, LLC","5100 Eastpark Blvd, Suite 210 0.0",...,8111.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14,NaN,550250114051048.0
1,1,945700,945700,945700,1299787308,4/28/20,563,PPP,TOTALMED INC.,221 W. COLLEGE AVE Floor 2,...,5613.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14,NaN,550870101001029.0
2,2,945701,945701,945701,1525827110,4/10/20,563,PPP,COURTESY CORPORATION,2700 National Drive Ste 100,...,7225.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14,NaN,550630105001009.0
3,3,945702,945702,945702,2572157107,4/10/20,563,PPP,AIR WISCONSIN AIRLINES LLC,"W6390 Challenger Dr, Suite 203 0.0",...,4811.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14,NaN,550870126042003.0
4,4,945703,945703,945703,3943447203,4/27/20,563,PPP,"FOREMOST FARMS USA, COOPERATIVE",10889 E PENNY LN,...,3115.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14,NaN,551110001031144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17349,17349,963048,963048,963048,9282518304,1/30/21,563,PPS,RESOURCE MACHINING & WELDING CORP.,W229N5065 Duplainville Rd,...,3327.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,NaN,551332033044000,551332033044000
17350,17350,963049,963049,963049,9361087700,5/1/20,563,PPP,"KORTENDICK HARDWARE, INC.",3806 DOUGLAS AVENUE,...,4441.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,NaN,551010014013000,551010014013000
17351,17351,963050,963050,963050,9411997102,4/15/20,563,PPP,HIGHLAND PARK FAMILY DENTISTRY SC,6101 W VLIET ST,...,6212.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,NaN,550790912001000,550790912001000
17352,17352,963051,963051,963051,9453378305,1/30/21,563,PPS,GRUETZ INC,E5690 County Road D,...,7225.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,NaN,550339703002036,550339703002036


Drop Extra FIPS Columns

In [23]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,945699,945699,945699,1040127103,4/9/20,563,PPP,"WISCONSIN KENWORTH, LLC","5100 Eastpark Blvd, Suite 210 0.0",...,999 Other,8111 Automotive Repair and Maintenance,8111.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,550250114051048.0
1,1,945700,945700,945700,1299787308,4/28/20,563,PPP,TOTALMED INC.,221 W. COLLEGE AVE Floor 2,...,999 Other,5613 Employment Services,5613.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,550870101001029.0
2,2,945701,945701,945701,1525827110,4/10/20,563,PPP,COURTESY CORPORATION,2700 National Drive Ste 100,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,550630105001009.0
3,3,945702,945702,945702,2572157107,4/10/20,563,PPP,AIR WISCONSIN AIRLINES LLC,"W6390 Challenger Dr, Suite 203 0.0",...,999 Other,4811 Scheduled Air Transportation,4811.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,550870126042003.0
4,4,945703,945703,945703,3943447203,4/27/20,563,PPP,"FOREMOST FARMS USA, COOPERATIVE",10889 E PENNY LN,...,311 Food,3115 Dairy Product Manufacturing,3115.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,551110001031144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17349,17349,963048,963048,963048,9282518304,1/30/21,563,PPS,RESOURCE MACHINING & WELDING CORP.,W229N5065 Duplainville Rd,...,332 Fabricated Metal Products,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,551332033044000
17350,17350,963049,963049,963049,9361087700,5/1/20,563,PPP,"KORTENDICK HARDWARE, INC.",3806 DOUGLAS AVENUE,...,999 Other,4441 Building Material and Supplies Dealers,4441.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,551010014013000
17351,17351,963050,963050,963050,9411997102,4/15/20,563,PPP,HIGHLAND PARK FAMILY DENTISTRY SC,6101 W VLIET ST,...,999 Other,6212 Offices of Dentists,6212.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,550790912001000
17352,17352,963051,963051,963051,9453378305,1/30/21,563,PPS,GRUETZ INC,E5690 County Road D,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,550339703002036


In [24]:
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
547,547,946246,946246,946246,4648077103,4/13/20,563,PPP,"MULTISTACK, LLC",CT Corporation System Inc 1010 Dale Street North,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,WISCONSIN DISTRICT OFFICE,d $2-5 million,100 to 249,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,NaN
1641,1641,947340,947340,947340,2925978305,1/21/21,563,PPS,JAKES ELECTRIC LLC,215 Front St,...,999 Other,2382 Building Equipment Contractors,2382.0,WISCONSIN DISTRICT OFFICE,c $1-2 million,50 to 99,"215 Front St, Clinton, WI",42.410926,-71.689619,NaN
1643,1643,947342,947342,947342,7736337004,4/8/20,563,PPP,"JAKES ELECTRIC, LLC",215 FRONT ST,...,999 Other,2382 Building Equipment Contractors,2382.0,WISCONSIN DISTRICT OFFICE,c $1-2 million,50 to 99,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,NaN
1712,1712,947411,947411,947411,6987067006,4/7/20,563,PPP,GILMAN CHEESE CORPORATION,300 RIVERSIDE DR,...,311 Food,3115 Dairy Product Manufacturing,3115.0,WISCONSIN DISTRICT OFFICE,c $1-2 million,100 to 249,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,NaN
2027,2027,947726,947726,947726,9739697001,4/9/20,563,PPP,"TIME TRANSPORT, INC.",5231 US Hwy 41,...,999 Other,4841 General Freight Trucking,4841.0,WISCONSIN DISTRICT OFFICE,c $1-2 million,50 to 99,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,NaN
2151,2151,947850,947850,947850,9170817005,4/9/20,563,PPP,"AMSTON SUPPLY, INC",7213 HWY 41 0.0,...,999 Other,4412 Other Motor Vehicle Dealers,4412.0,WISCONSIN DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7213 HWY 41 0.0, Caledonia, WI",25.763107,-80.312898,NaN
2659,2659,948358,948358,948358,2688497106,4/11/20,563,PPP,LISEC AMERICA INC,12571 OLIVER AVE S STE 100,...,327 Nonmetallic Mineral Product,3272 Glass and Glass Product Manufacturing,3272.0,WISCONSIN DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"12571 OLIVER AVE S STE 100, MILWAUKEE, WI",44.776233,-93.305704,NaN
3247,3247,948946,948946,948946,5637407007,4/6/20,563,PPP,"TRI STAR AUTO PARTS, INC.","320 10th Ave., PO Box 96",...,336 Transportation Equipment,3363 Motor Vehicle Parts Manufacturing,3363.0,WISCONSIN DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"320 10th Ave., PO Box 96, BALDWIN, WI",NaN,NaN,NaN
8676,8676,954375,954375,954375,4612047005,4/4/20,563,PPP,"ROUGHING IT IN STYLE, INC.",5074 HWY 51 NORTH,...,999 Other,4421 Furniture Stores,4421.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"5074 HWY 51 NORTH, HARSHAW, WI",34.398029,-89.927004,NaN
10304,10304,956003,956003,956003,4769537008,4/4/20,563,PPP,"MEYER SALES COMPANY, INC",600 North First Street,...,999 Other,4412 Other Motor Vehicle Dealers,4412.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"600 North First Street, CAMERON, WI",37.346986,-121.898560,NaN


Convert to File

In [25]:
fips_WI = fips_merge_drop


In [26]:
fips_WI.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WI_fips_scraped.csv") 

Review & Compare

In [27]:
fips_WI = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WI_fips_scraped.csv") 
fips_WI

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,945699,945699,945699,1040127103,4/9/20,563,PPP,"WISCONSIN KENWORTH, LLC",...,999 Other,8111 Automotive Repair and Maintenance,8111.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14
1,1,1,945700,945700,945700,1299787308,4/28/20,563,PPP,TOTALMED INC.,...,999 Other,5613 Employment Services,5613.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14
2,2,2,945701,945701,945701,1525827110,4/10/20,563,PPP,COURTESY CORPORATION,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14
3,3,3,945702,945702,945702,2572157107,4/10/20,563,PPP,AIR WISCONSIN AIRLINES LLC,...,999 Other,4811 Scheduled Air Transportation,4811.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14
4,4,4,945703,945703,945703,3943447203,4/27/20,563,PPP,"FOREMOST FARMS USA, COOPERATIVE",...,311 Food,3115 Dairy Product Manufacturing,3115.0,WISCONSIN DISTRICT OFFICE,e $5-10 million,500 or more,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17349,17349,17349,963048,963048,963048,9282518304,1/30/21,563,PPS,RESOURCE MACHINING & WELDING CORP.,...,332 Fabricated Metal Products,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,5.513320e+14
17350,17350,17350,963049,963049,963049,9361087700,5/1/20,563,PPP,"KORTENDICK HARDWARE, INC.",...,999 Other,4441 Building Material and Supplies Dealers,4441.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,5.510100e+14
17351,17351,17351,963050,963050,963050,9411997102,4/15/20,563,PPP,HIGHLAND PARK FAMILY DENTISTRY SC,...,999 Other,6212 Offices of Dentists,6212.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,5.507909e+14
17352,17352,17352,963051,963051,963051,9453378305,1/30/21,563,PPS,GRUETZ INC,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,WISCONSIN DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,5.503397e+14
